In [1]:
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split

import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchvision import transforms
import torchvision.models as models

In [2]:
df = pd.read_csv(r"D:\Datasets\100_Unique_QA_Dataset.csv")
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [3]:
#tokenize

def tokenize(text):
    text = text.lower()
    text = text.replace('?', '')
    text = text.replace("'", "")
    return text.split()

In [6]:
tokenize("Hello World")

['hello', 'world']

In [13]:
#vocab

vocab = {'<UNK>':0}

def build_vocab(row):
    
    tokenized_question = tokenize(row['question'])
    tokenized_answer = tokenize(row['answer'])

    merged_tokens = tokenized_question+tokenized_answer

    for token in merged_tokens:

        if token not in vocab:
            vocab[token] = len(vocab)

df.apply(build_vocab, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [14]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [15]:
def text_to_indices(text, vocab):

    indexed_text = []

    for token in tokenize(text):

        if token in vocab:
            indexed_text.append(vocab[token])
        else:
            indexed_text.append(vocab['<UNK>'])

    return indexed_text

text_to_indices("What is my name?", vocab)
        

[1, 2, 0, 0]

In [36]:
class customDataset(Dataset):

    def __init__(self, df, vocab):        
        self.df = df
        self.vocab = vocab

    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        
        num_question = text_to_indices(self.df.iloc[idx]['question'], self.vocab)
        num_answer = text_to_indices(self.df.iloc[idx]['answer'], self.vocab)

        return torch.tensor(num_question), torch.tensor(num_answer)

In [37]:
dataset = customDataset(df, vocab)

dataset[10]

(tensor([ 1,  2,  3,  4,  5, 53]), tensor([54]))

In [39]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [59]:
class rnn(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        #self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, 50)
        self.rnn = nn.RNN(50, 64, batch_first=True)
        self.fc = nn.Linear(64, vocab_size)
    
    def forward(self, question):
        embedded_question = self.embedding(question)
        hidden, final = self.rnn(embedded_question)
        output = self.fc(final.squeeze(0))

        return output

In [60]:
learning_rate = 0.01
epochs = 20

In [61]:
model = rnn(len(vocab))

In [62]:
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [67]:
# training loop

for epoch in range(epochs):

    total_loss = 0

    for question, answer in dataloader:

        optimizer.zero_grad()
        output = model(question)
        loss = criteria(output, answer[0])
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch: {epoch+1}/{epochs}, Loss: {total_loss}")

Epoch: 1/20, Loss: 0.03124911339546088
Epoch: 2/20, Loss: 0.02981320334947668
Epoch: 3/20, Loss: 0.028399812799762003
Epoch: 4/20, Loss: 0.02704977114626672
Epoch: 5/20, Loss: 0.025822699812124483
Epoch: 6/20, Loss: 0.02461407754162792
Epoch: 7/20, Loss: 0.02347431496309582
Epoch: 8/20, Loss: 0.022407240503525827
Epoch: 9/20, Loss: 0.02138960361480713
Epoch: 10/20, Loss: 0.020369093319459353
Epoch: 11/20, Loss: 0.019426154663960915
Epoch: 12/20, Loss: 0.018550663808127865
Epoch: 13/20, Loss: 0.01769791985861957
Epoch: 14/20, Loss: 0.016897971567232162
Epoch: 15/20, Loss: 0.016126015936606564
Epoch: 16/20, Loss: 0.015385285223601386
Epoch: 17/20, Loss: 0.01469853415619582
Epoch: 18/20, Loss: 0.014028345387487207
Epoch: 19/20, Loss: 0.013404157478362322
Epoch: 20/20, Loss: 0.012785566810634919


In [68]:
def predict(model, question, threshold=0.5):

  
  numerical_question = text_to_indices(question, vocab)

  question_tensor = torch.tensor(numerical_question).unsqueeze(0)
  output = model(question_tensor)

  
  probs = torch.nn.functional.softmax(output, dim=1)

  
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [70]:
predict(model, "What is the largest planet in our solar system")

jupiter
